In [1]:
from maml import *
import matplotlib.pyplot as plt

In [2]:
# maml = MAML(5, 0.01, 0.0001, 100, 100, 1000, -1, 1, eqname='burgers')
maml = MAML(5, 0.01, 0.0001, 0, 2, 1, low=-1, high=1, eqname='poisson')

train_loss, val_loss, val_ood_loss, nrmse, model = maml.train(1000, 1000, 5)

Initializing MAML-PINN model
Current device:  cuda
cuda
Finished initialization of MAML-PINN model
Start MAML training at iteration 0
[(0.2975355517357048, 0.6426448025595837), (-0.8677860529831849, -0.6510361708488535), (-0.43674561612997853, -0.20402468451939448), (0.7276316633336144, -0.6201224083093537), (-0.6448942280226355, -0.7012474400156419)]
[(1.163039302360969, -1.1654005652846529), (-1.1201188168188188, -1.1697721199875153), (-1.084434113566251, 1.3381620805489272), (1.0369430368497172, -1.4469926215438227), (1.3343718558930102, 1.069508922705026)]
Validation before training (0.298, 0.643)| Inner_loss_B: 0.0716 | Inner_loss_F: 0.0734 | Inner_loss: 0.7897 | NRMSE: 0.3265
Validation OOD before training (1.163, -1.165) | Inner_loss_B: 0.0997 | Inner_loss_F: 1.0728 | Inner_loss: 2.0699 | NRMSE: 0.8720
Step 1 | Inner_loss_B: 0.0645 | Inner_loss_F: 0.1506
Step 2 | Inner_loss_B: 0.0634 | Inner_loss_F: 0.1390
Step 3 | Inner_loss_B: 0.0624 | Inner_loss_F: 0.1320
Step 4 | Inner_loss_


KeyboardInterrupt



In [ ]:
# steps = [ 1000 / len(val_loss['inner_loss']) * i for i in range(len(val_loss['inner_loss']))]
# idx = [100 * i for i in range(10)]
# loss = train_loss['inner_loss_f'][::100]
# plt.plot(val_loss['inner_loss'], label='loss')
plt.plot(np.mean(nrmse['nrmse_val'], axis=1), label='Val.')
plt.plot(np.mean(nrmse['nrmse_val_ood'], axis=1), label='Val. OOD')
plt.legend()
plt.title('FO-MAML Val. Poisson')
# plt.plot( loss)
# plt.plot(nrmse['nrmse_val'])
# plt.plot(nrmse['nrmse_val_ood'])
# plt.plot(train_loss['inner_loss'])

In [ ]:
from train import *

In [ ]:
epochs = 500
lr = 0.001

In [ ]:
# loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse_set = train(epochs=epochs, lr=lr, i_size=100, b_size=100, f_size=1000, zero_shot=False, load=True, load_data='models/maml_burgers_5000.data', alpha_list=0.01 / np.pi, eqname='burgers')
loss_i, loss_b, loss_f, loss, model, val_loss, val_ood_loss, nrmse_set = train(epochs=epochs, lr=lr, i_size=0, b_size=2, f_size=10, zero_shot=False, load=True, load_data='maml.data', alpha_list=0.7, beta_list=-0.9, eqname='poisson')

In [ ]:
# loss_i_2, loss_b_2, loss_f_2, loss_2, model_2, val_loss_2, val_ood_loss_2, nrmse_set_2 = train(epochs=epochs, lr=lr, i_size=100, b_size=100, f_size=1000, zero_shot=False, load=False, load_data='maml.data', alpha_list=0.01 / np.pi, eqname='burgers')
loss_i_2, loss_b_2, loss_f_2, loss_2, model_2, val_loss_2, val_ood_loss_2, nrmse_set_2 = train(epochs=epochs, lr=lr, i_size=0, b_size=2, f_size=10, zero_shot=False, load=False, load_data='maml.data', alpha_list=0.7, beta_list=-0.9, eqname='poisson')

In [ ]:
plt.title('Loss Few shot')
plt.yscale('log')
plt.plot(loss, label='MAML')
plt.plot(loss_2, label='Random')
plt.legend()

In [ ]:
plt.title('Relative error Few shot')
plt.yscale('log')
plt.plot(nrmse_set, label='MAML')
plt.plot(nrmse_set_2, label='Random')
plt.legend()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)
test_x = np.linspace(-1, 1, num=100).reshape(-1, 1)
# test_alpha = np.full((100, 1), alpha[2])
# test_beta = np.full((100, 1), beta[2])
test_alpha = np.full((100, 1), 0.7)
test_beta = np.full((100, 1), 0.7)
# test_in = np.hstack((test_x, test_alpha, test_beta))
test_u = model_2(torch.Tensor(test_x).to(device))
X = test_x

Y = np.sin(test_alpha * X) + np.cos(test_beta * X) + 0.1 * X


In [ ]:
C = test_u.cpu().detach().numpy()
plt.figure(figsize=(10, 8))
plt.scatter(X, C, label='PINN')
plt.scatter(X, Y, label='Answer')
plt.legend()
plt.colorbar()

In [ ]:
plt.plot(loss)
plt.plot(loss_2)

In [ ]:
from burgers import *

In [ ]:
vtn = 101
vxn = 101
nu = 0.01 / np.pi
vx = np.linspace(-1, 1, vxn)
vt = np.linspace(0, 1, vtn)

vu = burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt)

x, t = np.meshgrid(vx, vt)
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)

plt.scatter(x, t, c=vu, cmap='seismic')
plt.colorbar()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current device:", device)

In [ ]:
vtn = 101
vxn = 101
nu = 0.01 / np.pi
vx = np.linspace(-1, 1, vxn)
vt = np.linspace(0, 1, vtn)
x, t = np.meshgrid(vx, vt)
x = x.reshape(-1, 1)
t = t.reshape(-1, 1)
alpha = np.full((x.shape), nu)
pred = model_2(torch.Tensor(np.hstack((x, t))).to(device)).detach().cpu().numpy()
# pred = model(torch.Tensor(np.hstack((x, t))).to(device)).detach().cpu().numpy()
truth = burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt).T.reshape(-1, 1)

In [ ]:
plt.scatter(x, t, c=pred, cmap='seismic')
plt.colorbar()